In [1]:
import torch

In [2]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

img = read_image("/Users/zjy/Downloads/511px-Grace_Hopper.jpg")

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

bow tie: 12.3%


In [3]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    param = model.state_dict()[param_tensor]
    try:
        print(param_tensor, "\t", param.size())
    except AttributeError:
        if isinstance(param, tuple):
            print(param_tensor, "\t", param[0].size())
        else:
            print(param_tensor, "\t", param)


Model's state_dict:
conv1.weight 	 torch.Size([64, 3, 7, 7])
bn1.weight 	 torch.Size([64])
bn1.bias 	 torch.Size([64])
bn1.running_mean 	 torch.Size([64])
bn1.running_var 	 torch.Size([64])
bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv1.weight 	 torch.Size([64, 64, 1, 1])
layer1.0.bn1.weight 	 torch.Size([64])
layer1.0.bn1.bias 	 torch.Size([64])
layer1.0.bn1.running_mean 	 torch.Size([64])
layer1.0.bn1.running_var 	 torch.Size([64])
layer1.0.bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv2.weight 	 torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight 	 torch.Size([64])
layer1.0.bn2.bias 	 torch.Size([64])
layer1.0.bn2.running_mean 	 torch.Size([64])
layer1.0.bn2.running_var 	 torch.Size([64])
layer1.0.bn2.num_batches_tracked 	 torch.Size([])
layer1.0.conv3.weight 	 torch.Size([256, 64, 1, 1])
layer1.0.bn3.weight 	 torch.Size([256])
layer1.0.bn3.bias 	 torch.Size([256])
layer1.0.bn3.running_mean 	 torch.Size([256])
layer1.0.bn3.running_var 	 torch.Size([256])
layer1.0.bn3.num

layer4.0.bn2.running_var 	 torch.Size([512])
layer4.0.bn2.num_batches_tracked 	 torch.Size([])
layer4.0.conv3.weight 	 torch.Size([2048, 512, 1, 1])
layer4.0.bn3.weight 	 torch.Size([2048])
layer4.0.bn3.bias 	 torch.Size([2048])
layer4.0.bn3.running_mean 	 torch.Size([2048])
layer4.0.bn3.running_var 	 torch.Size([2048])
layer4.0.bn3.num_batches_tracked 	 torch.Size([])
layer4.0.downsample.0.weight 	 torch.Size([2048, 1024, 1, 1])
layer4.0.downsample.1.weight 	 torch.Size([2048])
layer4.0.downsample.1.bias 	 torch.Size([2048])
layer4.0.downsample.1.running_mean 	 torch.Size([2048])
layer4.0.downsample.1.running_var 	 torch.Size([2048])
layer4.0.downsample.1.num_batches_tracked 	 torch.Size([])
layer4.1.conv1.weight 	 torch.Size([512, 2048, 1, 1])
layer4.1.bn1.weight 	 torch.Size([512])
layer4.1.bn1.bias 	 torch.Size([512])
layer4.1.bn1.running_mean 	 torch.Size([512])
layer4.1.bn1.running_var 	 torch.Size([512])
layer4.1.bn1.num_batches_tracked 	 torch.Size([])
layer4.1.conv2.weight 	 t